In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib as mpl
from matplotlib import pyplot as plt

from scipy.stats import zscore

from collections import defaultdict, Counter

import scipy.spatial as sp
import scipy.cluster.hierarchy as hc

from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

In [9]:
string = pd.read_csv("../data/9606.protein.links.v11.0.txt.gz",sep="\s")
string = string[string["combined_score"]>=800]

string["protein1_ensembl"] = string["protein1"].apply(lambda x: x[5:])
string["protein2_ensembl"] = string["protein2"].apply(lambda x: x[5:])


/Users/khu/anaconda3/envs/cenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [18]:
ensembl_gene_protein = pd.read_csv("../data/ensembl_gene_protein.txt",sep="\t")
ensembl_gene_protein = ensembl_gene_protein.dropna(subset=["Protein stable ID"])

protein_gene_map = dict(zip(ensembl_gene_protein["Protein stable ID"],ensembl_gene_protein["Gene stable ID"]))

string["protein1_gene"]  = string["protein1_ensembl"].apply(lambda x: protein_gene_map.get(x,np.nan))
string["protein2_gene"]  = string["protein2_ensembl"].apply(lambda x: protein_gene_map.get(x,np.nan))

string = string.dropna()

In [28]:
tss1kb_meth = pd.read_hdf("../data/CCLE_RRBS_TSS1kb_20181022.hdf",key="tss1kb_meth")
ccle_genex = pd.read_hdf("../data/CCLE_RNAseq_rsem_genes_tpm_20180929.hdf",key="ccle_genex")

